<a href="https://colab.research.google.com/github/Lightsource-Pris/Voting-Site/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import important libraries
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from urllib.parse import urlparse
from datetime import datetime
import whois
import joblib
from sklearn.model_selection import train_test_split

In [ ]:
# Load dataset
data = pd.read_csv('train_data.csv')
# Calculate 'domain_age' for each URL
def calculate_domain_age(url):
    domain = urlparse(url).netloc
    try:
        whois_info = whois.whois(domain)
        if isinstance(whois_info.creation_date, list):
            registration_date = min(whois_info.creation_date)
        else:
            registration_date = whois_info.creation_date

        current_date = datetime.now()
        age = (current_date - registration_date).days
        return age
    except:
        return 0  # Default age in case of an error

data['domain_age'] = data['url'].apply(calculate_domain_age)

# Calculate 'url_length' for each URL
data['url_length'] = data['url'].apply(len)

# Define a list of malicious keywords
malicious_keywords = ['login', 'cards', 'password', 'card']

# Calculate 'keyword_presence' for each URL
data['keyword_presence'] = data['url'].apply(lambda url: any(keyword in url for keyword in malicious_keywords))

# Save preprocessed dataset
data.to_csv('preprocessed_data.csv', index=False)

In [ ]:
data = pd.read_csv('preprocessed_data.csv')

# Define features
X = data[['domain_age', 'url_length', 'keyword_presence']]
y = data['type']

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)

# Create and train a RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save trained model to file
joblib.dump(model, 'phishing_model.pkl')

['phishing_model.pkl']